In [2]:
# import dependencies
from pprint import pprint
import pandas as pd
import json

In [7]:
raw_df = pd.read_csv('amazon_stock_data_2_year.csv')
df = raw_df[['t','n','v','vw','o','c','h','l']].rename(columns={'v':'volume',\
                                                            'vw':'volume weighted average price',\
                                                            'o':'open', 'c':'close',\
                                                            'h':'high', 'l':'low',\
                                                            't':'time','n':'transaction count'})
df.sort_values('time', ascending=True)
df.head()


,time,transaction count,volume,volume weighted average price,open,close,high,low
0,1604379600000,247963,4897747.0,3045.9706,3018.53,3048.41,3074.90,2980.9800
1,1604466000000,323719,6839041.0,3208.7783,3159.99,3241.16,3244.85,3139.7300
2,1604552400000,263106,5785685.0,3333.0865,3319.97,3322.00,3366.80,3288.8800
3,1604638800000,206123,4617250.0,3285.9249,3304.64,3311.37,3322.00,3232.0000
4,1604898000000,382356,7190406.0,3206.0707,3231.03,3143.74,3289.00,3112.1057
